# Feature preparation
Right here the features for the ml models are created and filtered. Then a csv with only the important features is saved.

In [43]:
import os
import pandas as pd
import pickle

from sklearn.preprocessing import LabelEncoder  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [44]:
path_trips_hexagons=os.path.join(os.getcwd(), "..", "data", "output", "Trips_Hexagons.csv")
path_hexagons=os.path.join(os.getcwd(), "..", "data", "output", "hexagons-dresden.csv")
path_trips = os.path.join(os.getcwd(), "..", "data", "output", "Trips.csv")
print(path_trips_hexagons)

C:\Users\lenovo\Documents\GitHub\AAA_Jupyter_Explorers\notebooks\..\data\output\Trips_Hexagons.csv


In [45]:
df = pd.read_csv(path_trips)
df.head()

,Unnamed: 0,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,...,trip_duration,idle_time,24_sum,6_sum,2_sum,1_sum,h3_hex_small_id_start,h3_hex_small_id_end,h3_hex_big_id_start,h3_hex_big_id_end
0,0,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,...,28.0,0.0,265,36,19,8,871f1b54bffffff,871f1b559ffffff,861f1b54fffffff,861f1b55fffffff
1,1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,...,10.0,0.0,265,33,17,17,871f1b460ffffff,871f1b464ffffff,861f1b467ffffff,861f1b467ffffff
2,2,0,True,0,0,2019-01-20 00:07:00,93440,10299584,1,51.042570,...,26.0,0.0,265,33,17,17,871f1b559ffffff,871f1b460ffffff,861f1b55fffffff,861f1b467ffffff
3,3,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,...,2.0,0.0,265,33,17,17,871f1b559ffffff,871f1b559ffffff,861f1b55fffffff,861f1b55fffffff
4,4,0,True,0,0,2019-01-20 00:07:00,93585,264575,5,51.071740,...,28.0,0.0,265,33,17,17,871f1b54bffffff,871f1b54bffffff,861f1b54fffffff,861f1b54fffffff


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567439 entries, 0 to 567438
Data columns (total 61 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             567439 non-null  int64  
 1   p_bike_racks_start     567439 non-null  int64  
 2   p_spot_start           567439 non-null  bool   
 3   p_booked_bikes_start   567439 non-null  int64  
 4   p_place_type_start     567439 non-null  int64  
 5   datetime_start         567439 non-null  object 
 6   b_number_start         567439 non-null  int64  
 7   p_uid_start            567439 non-null  int64  
 8   p_bikes_start          567439 non-null  int64  
 9   p_lat_start            567439 non-null  float64
 10  b_electric_lock_start  567439 non-null  bool   
 11  b_bike_type_start      567439 non-null  int64  
 12  p_name_start           567439 non-null  object 
 13  p_free_racks_start     567439 non-null  int64  
 14  b_lock_types_start     567439 non-nu

In [52]:
df.isnull().sum()

Unnamed: 0               0
p_bike_racks_start       0
p_spot_start             0
p_booked_bikes_start     0
p_place_type_start       0
                        ..
1_sum                    0
h3_hex_small_id_start    0
h3_hex_small_id_end      0
h3_hex_big_id_start      0
h3_hex_big_id_end        0
Length: 61, dtype: int64

## Drop not needed columns
Drop end information and not usefull columns.
The features are cleaned in three steps. <br>
- features_1 => drop not usefull columns 
- features_2 => drop end information
- features_3 => drop object types

In [54]:
print("Drop not int columns and not usefull")
# Unnamed: 0 => index column
# p_terminal_type_end => NaN values
features_1 = df.drop(["Unnamed: 0", 
                      #"p_terminal_type_start", 
                      "p_number_start", 
                      #"b_pedelec_battery_start"
                     ], axis=1)
print("Drop end information")
features_2 = features_1.drop(
    ["p_bike_racks_end",
     "p_spot_end", 
     "p_booked_bikes_end", 
     "p_place_type_end", 
     "datetime_end",
     "p_uid_end",
     "p_bikes_end",     
     "p_lat_end",
     "p_name_end",
     "p_free_racks_end",
     #"p_address_end",
     "p_number_end",
     "p_lng_end",
     "p_maintenance_end",
     "h3_hex_small_id_end",
     "h3_hex_big_id_end",
     "trip_duration"
     #"p_terminal_type_end", 
     #"p_bike_types_end"
    ], 
    axis=1
)
print("Drop object types...")
features_3 = features_2.drop(["datetime_start", 
                              "p_name_start", 
                              #"p_address_start", 
                              "b_lock_types_start",
                              "booking_date_start",
                              #"p_bike_types_start", 
                              #"MESS_DATUM"
                             ], axis=1)

Drop not int columns and not usefull
Drop end information
Drop object types...


### Dummy
Create Dummy variables for all booleans

In [55]:
p_spot_start = features_3["p_spot_start"].astype(int)
b_electric_lock_start = features_3["b_electric_lock_start"].astype(int)
p_maintenance_start = features_3["p_maintenance_start"].astype(int)
weekend = features_3["weekend"].astype(int)

le = LabelEncoder()
hexa_small = pd.Series(le.fit_transform(features_3["h3_hex_small_id_start"]))
hexa_big = pd.Series(le.fit_transform(features_3["h3_hex_big_id_start"]))

# weekend = p_df["Weekend"].astype(int)
features = features_3.drop(["p_spot_start", "b_electric_lock_start", "p_maintenance_start", "weekend", "h3_hex_small_id_start", "h3_hex_big_id_start"], axis=1)
features = pd.concat([features, p_spot_start, b_electric_lock_start, p_maintenance_start, weekend, hexa_small, hexa_big], axis=1)

### Feature Engineering

In [56]:
# features["XYZ"] = np.square(features["XXX"])

In [57]:
# Save features
features.to_csv(os.path.join(os.getcwd(), "..", "data", "output", "Features.csv"))

### Split Data

In [58]:
features_X = features.drop(["24_sum", "6_sum", "2_sum", "1_sum"], axis=1)
features_y = features["24_sum"] #TODO: make var
X_train, X_test, y_train, y_test = train_test_split(features_X, features_y, random_state=42, test_size=0.3)

### Scaling and PCA

In [59]:
# STANDARD SCALER
st_scaler = StandardScaler()
# fit scaler only on training set not on test set
st_scaler.fit(X_train)

# Save Scaler Object
obj = st_scaler
filename = "Standard_Scaler.pkl"
pickle.dump(obj, open(os.path.join(os.getcwd(), "..", "data", "output", "models", filename), "wb"))

X_train_scaled = st_scaler.transform(X_train)

In [68]:
# PCA
pca = PCA(n_components=20) # Number of components is set here
pca.fit(X_train_scaled)
pca_explained_variance = pca.explained_variance_ratio_
print("Var explained:", pca_explained_variance)
print("Sum var explained", sum(pca_explained_variance))

# Save PCA Object
obj = pca
filename = "PCA.pkl"
pickle.dump(obj, open(os.path.join(os.getcwd(), "..", "data", "output", "models", filename), "wb"))

Var explained: [0.15116754 0.09024424 0.07722063 0.07159344 0.06053688 0.05859782
 0.05745243 0.05158906 0.05056532 0.04886894 0.03663008 0.03028615
 0.0299577  0.02838975 0.02822204 0.0268716  0.01775359 0.01640399
 0.01448348 0.01402899]
Sum var explained 0.9608636938598711
